### Test of CV functionality. 

Most storage related tests are nosetests now. So look in `testcollectivevariable.py` please.

In [1]:
import openpathsampling as paths
import numpy as np
import openpathsampling.engines.openmm as peng

Load first frame from test pdb.

In [2]:
template = peng.snapshot_from_pdb('../resources/AD_initial_frame.pdb')

Create a simple CV without an underlying function.

In [3]:
cv0 = paths.CollectiveVariable('func0')

Create a complicated function.

In [4]:
center = 1
def dist(snapshot, center, np):
    return np.sum(snapshot.coordinates._value[0]) - center

Create collective variable from this function. Note that you have to specify `center` and `np` to make this work

In [5]:
cv1 = paths.FunctionCV('func1', dist, center=center, np=np, cv_time_reversible=False).with_diskcache()
cv2 = paths.FunctionCV('func2', dist, center=center, np=np, cv_wrap_numpy_array=True).with_diskcache()
cv3 = paths.FunctionCV('func3', dist, center=center, np=np, cv_wrap_numpy_array=True, cv_time_reversible=True).with_diskcache()

Create storage to test save and load.

In [6]:
#! lazy
storage = paths.Storage('can_be_deleted.nc', mode='w')
print storage.snapshots.save(template)

284548731370429831383829540429094715404


Save CV

In [7]:
#! lazy
print storage.save([cv0, cv1, cv2, cv3])

[(store.attributes[Attribute] : 4 object(s), 0, 284548731370429831383829540429094715406L), (store.attributes[Attribute] : 4 object(s), 0, 284548731370429831383829540429094715408L), (store.attributes[Attribute] : 4 object(s), 0, 284548731370429831383829540429094715410L), (store.attributes[Attribute] : 4 object(s), 0, 284548731370429831383829540429094715412L)]


In [8]:
storage.cvs.index

{284548731370429831383829540429094715406L: 0,
 284548731370429831383829540429094715408L: 1,
 284548731370429831383829540429094715410L: 2,
 284548731370429831383829540429094715412L: 3}

Set the CV value for the storage.template

In [9]:
cv0[template] = 10.0

and create a storage for it

In [10]:
storage.cvs.add_diskcache(cv0, allow_incomplete=True)

And we should have a float store

In [11]:
storage.cvs.sync(cv0)

In [12]:
assert(cv0._store_dict.value_store.vars['value'][:] == [10.0])

Test function for reversed template.

In [13]:
dd = dist(template.reversed, center, np)
print dd

-1.08320000023


Create another CV. This time using the `from_template` function.

In [14]:
print cv0([template, template])
print cv1([template, template])

[10.0, 10.0]
[-1.0832000002264977, -1.0832000002264977]


In [15]:
print type(cv0([template, template]))
print type(cv0([template, template])[0])
print type(cv1([template, template]))
print type(cv1([template, template])[0])

<type 'list'>
<type 'float'>
<type 'list'>
<type 'numpy.float64'>


In [16]:
#! skip
print storage.cvs.variables['json'][:]

[ u'{"_cls":"CollectiveVariable","_dict":{"cv_time_reversible":false,"name":"func0"}}'
 u'{"_cls":"FunctionCV","_dict":{"name":"func1","f":{"_marshal":"YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTQtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB","_module_vars":[],"_global_vars":[]},"cv_time_reversible":false,"cv_wrap_numpy_array":false,"cv_requires_lists":false,"cv_scalarize_numpy_singletons":false,"kwargs":{"np":{"_import":"numpy"},"center":1}}}'
 u'{"_cls":"FunctionCV","_dict":{"name":"func2","f":{"_marshal":"YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTQtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB","_module_vars":[],"_global_vars":[]},

In [17]:
cv0j = storage.cvs.vars['json'][0]
cv1j = storage.cvs.vars['json'][1]
cv2j = storage.cvs.vars['json'][2]
cv3j = storage.cvs.vars['json'][3]

In [18]:
res = cv0j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv1j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv2j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv3j([template, template, template])
assert(res[0] == res[1] == res[2])

In [19]:
t = paths.Trajectory([template] * 4)

In [20]:
storage.save(t)

(store.trajectories[Trajectory] : 1 object(s),
 1,
 284548731370429831383829540429094715480L)

In [21]:
def ff(t, cv3):
    return max(cv3(t))

In [22]:
a = paths.netcdfplus.FunctionAttribute('max_cv3', paths.Trajectory, ff, cv3=cv3)

In [23]:
storage.attributes.save(a)

284548731370429831383829540429094715482L

In [24]:
a(t)

-1.0832000002264977

In [25]:
p = paths.netcdfplus.LoaderProxy(storage.trajectories, t.__uuid__)

In [26]:
a(paths.netcdfplus.LoaderProxy(storage.trajectories, t.__uuid__))

-1.0832000002264977

In [27]:
storage.trajectories.add_attribute(paths.netcdfplus.ValueStore, a, t, allow_incomplete=False)

store.trajectories_max_cv3[None/ANY] : (not created)

In [28]:
ats = storage.trajectories.attribute_list[a]

In [29]:
print ats.vars['value'][:]
if ats.allow_incomplete:
    print ats.vars['index'][:]

[-1.0832]


In [30]:
a(t)

-1.0832000002264977

In [31]:
print a._store_dict.value_store.vars['value'][:]

[-1.0832]


In [32]:
a._store_dict.value_store.variables['value']

<type 'netCDF4._netCDF4.Variable'>
float64 trajectories_max_cv3_value(trajectories)
    var_type: numpy.float64
unlimited dimensions: trajectories
current shape = (1,)
filling on, default _FillValue of 9.96920996839e+36 used

In [33]:
cv3._store_dict.value_store.variables['value']

<type 'netCDF4._netCDF4.Variable'>
float64 cv3_value(snapshots)
    var_type: numpy.float64
unlimited dimensions: snapshots
current shape = (1,)
filling on, default _FillValue of 9.96920996839e+36 used

In [34]:
storage.trajectories.sync_attribute(a)

In [35]:
storage.attributes.set_cache_store(a)

In [36]:
storage.attributes.has_cache(a)

True

In [37]:
storage.attributes.sync(a)

In [55]:
storage.close()

In [62]:
storage = paths.Storage('can_be_deleted.nc', mode='r')

In [63]:
storage.attributes[4]

In [58]:
storage.close()

In [59]:
import openpathsampling as paths
import numpy as np
import openpathsampling.engines.openmm as peng

In [60]:
st = paths.Storage('../delete.nc', mode='r')

In [61]:
st.attributes

store.attributes[Attribute] : 3 object(s)

In [3]:
st._obj_store[paths.CollectiveVariable]

store.attributes[Attribute] : 3 object(s)

In [3]:
cv = st.cvs[0]

In [4]:
max_vals = [
    paths.netcdfplus.FunctionAttribute(
        cv.name + '_max', 
        paths.Trajectory,
        lambda t, cv: max(cv(t)), cv=cv
    ).with_diskcache() for cv in st.cvs]

In [5]:
for c in paths.CollectiveVariable.descendants():
    st._obj_store[c] = st.cvs
st._obj_store[paths.CollectiveVariable] = st.cvs

In [6]:
st._obj_store[paths.CollectiveVariable]

store.cvs[BaseSnapshot] : 3 object(s)

In [7]:
map(st.attributes.save, max_vals)

[272535980374264538949855631713773813822L,
 272535980374264538949855631713773813826L,
 272535980374264538949855631713773813830L]

In [8]:
st.attributes.name_idx

{'opA_max': {1}, 'opB_max': {0}, 'opC_max': {2}}

In [9]:
st.attributes['opA_max']

In [10]:
st.attributes[0]

In [11]:
st.attributes[0](st.trajectories)

[0.367531418800354,
 0.8562381863594055,
 0.8126517534255981,
 0.8433051705360413,
 0.8689201474189758,
 1.0538431406021118,
 0.367531418800354,
 1.0822958946228027,
 1.2032464742660522,
 0.8319146037101746,
 0.8430009484291077,
 0.8758599758148193,
 1.0448485612869263,
 0.8692688345909119,
 0.8562381863594055,
 0.8689201474189758,
 0.8126517534255981,
 1.1879923343658447,
 0.8692688345909119,
 0.8368529081344604,
 0.8512461185455322,
 0.875219464302063,
 0.8360109925270081,
 0.8126517534255981,
 0.8598640561103821,
 0.8086681962013245,
 1.0538431406021118,
 1.0743924379348755,
 0.8598640561103821,
 0.8086681962013245,
 0.367531418800354,
 0.8598640561103821,
 0.3687124252319336,
 1.1011096239089966,
 0.8430009484291077,
 0.8430009484291077,
 0.8931102752685547,
 0.8598640561103821,
 0.8178797960281372,
 0.8919672966003418,
 0.8433051705360413,
 1.0538431406021118,
 0.8032494783401489,
 0.8689201474189758,
 0.8178797960281372,
 0.8537729382514954,
 0.8433051705360413,
 0.85623818635940